In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder

In [75]:
train=pd.read_csv("/content/train.csv")
test=pd.read_csv("/content/test.csv")

In [76]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [77]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [78]:
train.isnull().sum().sort_values(ascending=False)

,0
PoolQC,1453
MiscFeature,1406
Alley,1369
Fence,1179
MasVnrType,872
...,...
MoSold,0
YrSold,0
SaleType,0
SaleCondition,0


In [79]:
train.drop('PoolQC',axis=1,inplace=True)
train.drop('MiscFeature',axis=1,inplace=True)
train.drop('Alley',axis=1,inplace=True)
train.drop('Fence',axis=1,inplace=True)
train.drop('MasVnrType',axis=1,inplace=True)
train.drop('FireplaceQu',axis=1,inplace=True)


In [80]:
test.isnull().sum().sort_values(ascending=False)

,0
PoolQC,1456
MiscFeature,1408
Alley,1352
Fence,1169
MasVnrType,894
...,...
EnclosedPorch,0
MiscVal,0
MoSold,0
YrSold,0


In [81]:
test.drop('PoolQC',axis=1,inplace=True)
test.drop('Alley',axis=1,inplace=True)
test.drop('MasVnrType',axis=1,inplace=True)
test.drop('MiscFeature',axis=1,inplace=True)
test.drop('Fence',axis=1,inplace=True)
test.drop('FireplaceQu',axis=1,inplace=True)

In [82]:
cat_cols=train.select_dtypes(include=['object']).columns

In [83]:
from numpy._core.defchararray import encode
for col in cat_cols:
  train[col]=train[col].astype(str)
  test[col]=test[col].astype(str)


In [84]:
encoder=OrdinalEncoder(
    handle_unknown='use_encoded_value',
    unknown_value=-1
)

In [85]:
train[cat_cols]=encoder.fit_transform(train[cat_cols])

In [86]:
test[cat_cols]=encoder.transform(test[cat_cols])

In [87]:
for col in test.columns:
  if test[col].dtype=='int64' or test[col].dtype=='float64':
    test[col]=test[col].fillna(test[col].median())

In [88]:
for col in test.columns:
  if test[col].dtype=="object":
    test[col]=test[col].fillna(test[col].mode()[0])

In [89]:
for col in train.columns:
  if train[col].dtype=='int64' or train[col].dtype=='float64':
    train[col]=train[col].fillna(train[col].median())

In [90]:
for col in train.columns:
  if train[col].dtype=="object":
    train[col]=train[col].fillna(train[col].mode()[0])

In [91]:
train.isnull().sum()

,0
Id,0
MSSubClass,0
MSZoning,0
LotFrontage,0
LotArea,0
...,...
MoSold,0
YrSold,0
SaleType,0
SaleCondition,0


In [92]:
test.isnull().sum()

,0
Id,0
MSSubClass,0
MSZoning,0
LotFrontage,0
LotArea,0
...,...
MiscVal,0
MoSold,0
YrSold,0
SaleType,0


In [98]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [94]:
X=train.drop(['SalePrice'],axis=1)
y=train['SalePrice']

In [96]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [101]:
rf=RandomForestRegressor()

In [102]:
model=rf.fit(X_train,y_train)

In [103]:
y_pred=model.predict(X_test)

In [105]:
r2=r2_score(y_test,y_pred)

In [107]:
r2

0.8901394906757775